In [ ]:
conda install nglview -c conda-forge

In [ ]:
#conda install pytraj -c conda-forge

In [ ]:
conda install -c conda-forge openmm==7.5.0


In [14]:
import nglview as nv
#import pytraj as pt
import random

from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout

import xml.etree.ElementTree as ElementTree

In [8]:
def load_sequence(path):
    with open(path) as f:
        fasta = f.read().strip()
    return fasta.split('\n')[-1]
    
seq = load_sequence('proteins/6ct4.fasta')
seq

'PMKKLKLALRLAAKIAPVW'

In [9]:
tree = ElementTree.parse('forcefields/amber99sb.xml').getroot()
atoms = {}
for c in tree[1]:
    atoms[c.attrib['name']] = [a.attrib['name'] for a in c.findall('Atom')]


In [10]:
mappings = 'ala:A|arg:R|asn:N|asp:D|cys:C|gln:Q|glu:E|gly:G|his:H|ile:I|leu:L|lys:K|met:M|phe:F|pro:P|ser:S|thr:T|trp:W|tyr:Y|val:V'.upper().split('|')
l2aa = dict([m.split(':')[::-1] for m in mappings])

In [11]:
def write_randpos_pdb(path, aa_sequence):
    data = ''
    serial = 1
    for residue_num, residue_letter in enumerate(aa_sequence):
        residue = l2aa[residue_letter]
        atom_names = atoms[residue].copy()
        if residue_num == 0:
            atom_names += ['H2', 'H3']
        if residue_num == len(aa_sequence) - 1:
            atom_names += ['OXT']
        for name in atom_names:
            rand = lambda s=0.1, off=0: (random.random() - 0.5) * 2 * s + off
            if name == 'CA':
                x, y, z = rand(s=0.01, off=residue_num), rand(s=0.01), rand(s=0.01)
            else:
                x, y, z = rand(off=residue_num), rand(), rand()
            
            if len(name) == 3:
                name = ' ' + name
            data += f'{"ATOM":6}{serial:5} {name:^4} {residue:3} {"A":1}{residue_num+1:4}    {x:8.3f}{y:8.3f}{z:8.3f}{1:6.2f}{0:6.2f}           {name.strip()[0]:2}{"":2}\n'
            serial += 1
    with open(path, 'w') as file:
        file.write(data)

write_randpos_pdb('proteins/test.pdb', seq)

In [12]:
from pdbfixer import PDBFixer
from simtk.openmm.app import PDBFile

def pdbfixerFunc(out_path,input_path):
    
    fixer = PDBFixer(filename=input_path)
    fixer.findMissingResidues()
    fixer.findNonstandardResidues()
    fixer.replaceNonstandardResidues()
    fixer.removeHeterogens(True)
    fixer.findMissingAtoms()
    fixer.addMissingAtoms()
    fixer.addMissingHydrogens(7.0)
    #fixer.addSolvent(fixer.topology.getUnitCellDimensions())
    PDBFile.writeFile(fixer.topology, fixer.positions, open(out_path, 'w'))
    
    
pdbfixerFunc('proteins/FixedFunctionOutput6ct4.pdb','proteins/test.pdb')
    

In [13]:
pdb = PDBFile('proteins/FixedFunctionOutput6ct4.pdb')
#forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
#forcefield = ForceField('amber99sb.xml')
forcefield=ForceField('forcefields/amber99sb.xml','C:/Users/hp/anaconda3/envs/tensorflow/Lib/site-packages/simtk/openmm/app/data/tip3p.xml')

print(pdb.topology)

modeller = Modeller(pdb.topology, pdb.positions)
modeller.addHydrogens()

system = forcefield.createSystem(modeller.topology, constraints=HBonds)
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)

simulation.minimizeEnergy()

steps = 100000
simulation.reporters.append(PDBReporter('proteins/output.pdb',  5))
simulation.reporters.append(StateDataReporter(stdout, 1, potentialEnergy=True, temperature=True))


<Topology; 1 chains, 19 residues, 335 atoms, 338 bonds>


In [12]:

simulation.step(5)

#"Potential Energy (kJ/mole)","Temperature (K)"
10153809149952.0,694640716686.0358
1900165070848.0,121163701824278.94
28515795730432.0,550675716262.8588
5929141010432.0,70982192987485.23
16114819006464.0,92755850436302.19
